# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Первичное ознакомление  с данными

In [1]:
import pandas as pd #импортируем библиотеку pandas

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv('/datasets/data.csv') #введем переменную для нашего датасета и пропишем путь для открытия таблицы с данными

In [4]:
df.info()# получим общую информацию о таблице

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [5]:
df.head() #посмотрим, что из себя представляет таблица "с головы"

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [6]:
df.tail() #'с хвоста' тоже не будет лишним:)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля
21524,2,-1984.507589,40,среднее,1,женат / замужем,0,F,сотрудник,0,82047.418899,на покупку автомобиля


**Вывод**
   

В представленной таблице содержатся данные о клиентах, заключивших с банком кредитный договор. Информация о каждом должнике хранится в отдельной строчке на 12 столбцах. При исследовании общих сведений об объекте DataFrame после вызова метода info(), было отмечено, что количество строк в столбцах "days_employed" и "total_income" отличается от количества в других столбцах, при этом между собой данные столбцы по количеству строк одинаковы. Кроме того, как следует из текста задания, в "days_employed" отображены  суммы общего трудового стажа в днях. Из этого следует, что данная информация должна быть представлена в целочисленном виде. Между тем, в указанном столбце сведения имеют формат float64, то есть в виде десятичной дроби.

## Предобработка данных

### Обработка пропусков

In [7]:
df[df['days_employed'].isnull()].head() #вызываем метод isna(), чтобы посмотреть в каких столбцах присутствуют пропуски

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


In [8]:
#пропущенные значения содеражатся в столбцах "days_employed" и "total_income"
len(df[df['days_employed'].isna()]) #посчитаем количество строк, в которых содержаться в пропуски

2174

In [9]:
# В таблице присутствуют более двух тысяч строк с пропущенными значениями, что составляет примерно 10% от общего количества.
df_income_median = df['total_income'].median() #посчитали медиану и сохранили ее в отдельной переменной

In [10]:
df['days_employed'].median()

-1203.369528770489

In [11]:
data_transformed = df.groupby('income_type')['total_income'].transform('mean') #сгруппировали таблицу для замены пропущенных значений на среднее по группе
 
df['total_income'] = df['total_income'].fillna(data_transformed)

In [12]:
# признаться, тут я просто считерил:) этот способ описывал наш преподаватель в  Slack, как возможный вариант решения 

In [13]:
df['days_employed'] = df['days_employed'].fillna(0) #заполним пропуски нулями, так как нам не нужен данный столбец

In [14]:
df.info() #убедимся, что пропусков больше нет

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [15]:
df['children'].value_counts() #теперь проверим, есть ли в столбцах пропуски в виде строковых "None" или " " 

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [16]:
df['dob_years'].value_counts() # 101 должник, которому 0 лет

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

In [17]:
df['dob_years'].median() #чтобы заменить значение возраста 0 лет, посчитаем медиану и подставим ее значение вместо нуля

42.0

In [18]:
df.loc[df['dob_years'] == '0', 'dob_years' ] = '42' # поменяем значение в столбце с возрастом

In [19]:
df['dob_years'].mean() #уточнил этот вопрос ниже. Разбил данные по возрастным группам. Теперь замена на медиану более-менее обоснована   

43.29337979094077

In [20]:
def age_group(age):
    if age <=18:
        return 'дети'
    if age <= 64:
        return 'взрослые'
    return 'пенсионеры'

In [21]:
df['age_group'] = df['dob_years'].apply(age_group)
df['age_group'].value_counts()

взрослые      20525
пенсионеры      899
дети            101
Name: age_group, dtype: int64

In [22]:
data_adult = df[df['age_group'] == 'взрослые']
data_adult['dob_years'].median()

42.0

In [23]:
# сделал)

In [24]:
df['education'].value_counts() # большое количество дубликатов с учетом регистра

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

In [25]:
df['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [26]:
df['gender'].value_counts() # какой-то ХNA...

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [27]:
df[df['gender'] == 'XNA']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
10701,0,-2358.600502,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.157261,покупка недвижимости,взрослые


In [28]:
df['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
студент                1
в декрете              1
Name: income_type, dtype: int64

In [29]:
df['debt'].value_counts()

0    19784
1     1741
Name: debt, dtype: int64

**Вывод**

В таблице было выявлено свыше двух тысяч строк с пропущенными значениями, которые содержались в столбцах "days_employed" и "total_income". При этом в каждом из этих столбцов количество пропусков  было одинаково. 

При анализе было отмечено, что пропуски встречаются в строках с абсолютно разными значениями, то есть нет никакой взаимосвязи между внесенными данными и самим пропуском. Строки со значением  "NaN" появляются в самом начале таблицы (строка 12) и присутствуют на протяжении всего объекта DataFrame вплоть до самых последних. Таким образом, данные пропуски являются полностью случайными. 

Обнаруженную абривиатуру XNA в столбце  с полом клиентов считаем полностью случайным пропуском в категориальной переменной, однако заполнить его не представляется возможным ввиду отсутствия данных об имени клиента.  

Сложно высказаться однозначно о причинах появления пропусков, поскольку в них нет никакой взаимосвязи. Возможно, данная таблица была составлена на основании анкет-заявок на получение кредита с сайта банка, в которых поля с информацией о трудовом стаже и доходе заявителя не являются обязательными для заполнения. 

В своей совокупности, пропуски составляют приблизительно 10% от общего количества строк, поэтому мы не можем просто удалить эти строки методом dropna(), так как это может привести к искаженным результатам исследования. Поскольку пропуски содержаться на месте количественных переменных, было принято решение заменить их на значение медианы, используя соответствующий метод median(). Применять метод mean() для замены значений на среднее арифметическое полагаем нецелесообразным, поскольку достаточно всего несколько должников, имеющих значительный доход, чтобы средней уровень дохода всех клиентов значительно превысил их самый распространенный уровень.

Касательно попусков в столбце "days_employed" было принято решение заменить их на значение "0", поскольку для проведения анализа  и ответа на поставленные вопросы, у нас нет необходомости  оперировать сведениями, содержащимися в данном столбце.

### Замена типа данных

In [30]:
df['total_income'] = df['total_income'].astype('int') #приводим значение дохода должников к целочисленному значению

In [31]:
df['days_employed'] = df['days_employed'].astype('int') #для красоты сделаем это и для столбца с трудовым стажем

In [32]:
df.loc[df['gender'] == 'M', 'gender'] = 'мужской' # сведения о поле приведем к более понятной форме

In [33]:
df.loc[df['gender'] == 'F', 'gender'] = 'женский'

In [34]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
0,1,-8437,42,высшее,0,женат / замужем,0,женский,сотрудник,0,253875,покупка жилья,взрослые
1,1,-4024,36,среднее,1,женат / замужем,0,женский,сотрудник,0,112080,приобретение автомобиля,взрослые
2,0,-5623,33,Среднее,1,женат / замужем,0,мужской,сотрудник,0,145885,покупка жилья,взрослые
3,3,-4124,32,среднее,1,женат / замужем,0,мужской,сотрудник,0,267628,дополнительное образование,взрослые
4,0,340266,53,среднее,1,гражданский брак,1,женский,пенсионер,0,158616,сыграть свадьбу,взрослые


**Вывод**

В столбце с доходом содержалась информация о доходе должников  в виде десятичной дроби до миллионой доли. Если речь идет не о криптовалюте, то эта форма является неправильной. Возможно, вводимую сумму встроенный на сайте банка конвертер автоматически переводит в доллары или евро. В этой же форме данные так и храняться в банковской базе. Однако, при извлечении информации из базы программа снова переводит в рубли по действующему курсу, при этом неверно задан формат выводимых данных.   

Чтобы привести сведения к целочисленному виду, был использован метод astype() с аргументом 'int'. Также в таблице была изменена информация о поле путем метода loc[] и логической индексации. 

### Обработка дубликатов

In [35]:
df.duplicated().sum() #посчитаем количество дубликатов

54

In [36]:
'{:.2%}'.format(df.duplicated().sum() / len(df)) #высчитаем процентную долю дубликатов 

'0.25%'

In [37]:
df = df.drop_duplicates().reset_index(drop = True) #изменим наш DataFrame и удалим все дубликаты с заменой индексов 

In [38]:
df.duplicated().sum() # дубликатов больше нет

0

In [39]:
df['education'] = df['education'].str.lower() #уберем дубликаты, приведя строки к нижнему регистру

In [40]:
df['education'].value_counts()

среднее                15188
высшее                  5251
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

**Вывод**

Для поиска и подчета дубликатов мы применили методы duplicated() и sum(), после чего удалили их, используя drop_duplicates(). Чтобы заполнить оставшиеся пустые строки, был использован метод reset_index() с аргументом "drop = True". Данный метод позволяет переписать таблицу без пустых строк. Дубликаты могли образоваться вследствие программного сбоя, а также по причине повторного нажатия клавиши "оформить заявку".
Дубликаты с учетом регистра в колонке "образование" были устранены путем приведения текста к нижнему регистру методом str.lower()

### Лемматизация

In [41]:
from pymystem3 import Mystem # импортируем библиотеку для стемминга
m = Mystem()

In [42]:
df['lemma_purpose'] = df['purpose'].apply(m.lemmatize) #добавим новый столбец с лемматезированными значениями

In [43]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,lemma_purpose
0,1,-8437,42,высшее,0,женат / замужем,0,женский,сотрудник,0,253875,покупка жилья,взрослые,"[покупка, , жилье, \n]"
1,1,-4024,36,среднее,1,женат / замужем,0,женский,сотрудник,0,112080,приобретение автомобиля,взрослые,"[приобретение, , автомобиль, \n]"
2,0,-5623,33,среднее,1,женат / замужем,0,мужской,сотрудник,0,145885,покупка жилья,взрослые,"[покупка, , жилье, \n]"
3,3,-4124,32,среднее,1,женат / замужем,0,мужской,сотрудник,0,267628,дополнительное образование,взрослые,"[дополнительный, , образование, \n]"
4,0,340266,53,среднее,1,гражданский брак,1,женский,пенсионер,0,158616,сыграть свадьбу,взрослые,"[сыграть, , свадьба, \n]"


In [44]:
def purpose_group(row): #сделаем функцию для категоризации по целям кредитования
    purpose = row['lemma_purpose']
    if 'свадьба' in purpose:
        return 'свадьба'
    if 'образование' in purpose: 
        return 'образование' 
    if 'автомобиль' in purpose:
        return 'автомобиль'
    if 'жилье' in purpose:
        return 'жилье'
    if 'недвижимость' in purpose:
        return 'жилье'


In [45]:
df['purpose_group'] = df.apply(purpose_group, axis=1) #применим функцию, создав новый столбец

In [46]:
df['purpose_group'].value_counts()

жилье          10814
автомобиль      4308
образование     4014
свадьба         2335
Name: purpose_group, dtype: int64

Для лемматизации информации о целях получения кредита был использована библиотека pymystem3. Ко всему столбцу был применен метод lemmatize c образованием нового столбца с леммами целей. Далее информация была разбита на 4 категории: "автомобиль", "жилье", "свадьба" и "образование".

### Категоризация данных

In [47]:
education_dict = df[['education', 'education_id']] #сделаем словарь для образования
education_dict = education_dict.drop_duplicates().reset_index(drop=True)
print(education_dict)

             education  education_id
0               высшее             0
1              среднее             1
2  неоконченное высшее             2
3            начальное             3
4       ученая степень             4


In [48]:
family_dict = df[['family_status', 'family_status_id']] #сделаем словарь для семейного положения
family_dict = family_dict.drop_duplicates().reset_index(drop=True)
print(family_dict)

           family_status  family_status_id
0        женат / замужем                 0
1       гражданский брак                 1
2         вдовец / вдова                 2
3              в разводе                 3
4  Не женат / не замужем                 4


In [49]:
def official_marriage_count(row): #сделаем функцию для раздела по наличию официального бракосочетания. 
    status = row['family_status_id']
    if status == 0:
        return 'оф. брак'
    else:
        return 'не сост в оф. браке'

In [50]:
df['marriage'] = df.apply(official_marriage_count, axis=1)

In [51]:
df['marriage'].value_counts()

оф. брак               12344
не сост в оф. браке     9127
Name: marriage, dtype: int64

In [52]:
#создадим функцию для раздела по количеству детей
def child_group(row):
    child = row['children']
    if 1 <= child <= 2:
        return 'есть дети'
    if child >= 3:
        return 'многодетные'
    if child == 0 :
        return 'нет детей'   

In [53]:
df['children_group'] = df.apply(child_group, axis=1)

In [54]:
df['children_group'].value_counts() #уточнил по многодетным

нет детей      14107
есть дети       6861
многодетные      456
Name: children_group, dtype: int64

In [55]:
def income_group_count(row):#функция для категоризации по уровню дохода
    income = row['total_income']
    try:
        if income > 1000000:
            return 'состоятельные'
        if 500000 < income <= 1000000:
            return 'верхний средний класс'
        if 250000 < income <= 500000:
            return 'средний класс'
        if 100000 < income <= 250000:
            return 'предсредний класс'
        if 36000 < income <= 100000:
            return 'выше бедности'
        if 0 < income <= 36000:
            return 'бедные'
    except:
        return 'невозможно определить'    
    
    
    

In [56]:
df['income_group'] = df.apply(income_group_count, axis=1)

In [57]:
df['income_group'].value_counts()

предсредний класс        14195
выше бедности             4388
средний класс             2591
верхний средний класс      197
бедные                      75
состоятельные               25
Name: income_group, dtype: int64

**Вывод**

Были созданы несколько словарей: по образованию, по семейному положению. Для ответа на вопросы информацию о детях разделили на две категории: есть дети или нет, а также по наличию официального брака или его его отсутствию. Считаем, что именно такая категоризация соответствует целям исследования. Информация об уровне дохода также была категоризирована путем создания функции, которая была применена к столбцу с доходом.  

## Ответы на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [58]:
len(df[df['debt'] == 1]) #посчитаем сколько у нас должников есть вообще

1741

In [59]:
#создадим функцию, которая будет возвращать количество должников в зависимости от количества детей
def children_debt_count(df, children, debt):
    debt_list = df[(df['children_group'] == children)&(df['debt'] == debt)]
    debt_list_count = debt_list.loc[:, 'debt'].count()
    return debt_list_count

In [60]:
#создадим переменные, к которых будут храниться требуемые нам цифры

#переменные для подсчета должников с детьми:
children_debt_sum = children_debt_count(df, "есть дети", 1) # здесь кол-во должников с детьми и с просрочкой
children_debt_total = children_debt_count(df, "есть дети", 0) + children_debt_count(df, "есть дети", 1) #здесь общее кол-во клиентов с детьми

#переменные для подсчета должников без детей:
childfree_debt_sum = children_debt_count(df, "нет детей", 1) #кол-во должников без детей, но с просрочкой
childfree_debt_total = children_debt_count(df, "нет детей", 0) + children_debt_count(df, "нет детей", 1) #общее кол-во клиентов без детей

#переменные для подсчета многодетных должников:
large_family_debt_sum = children_debt_count(df, "многодетные", 1) #кол-во многодетных должников с просрочкой
large_family_debt_total = children_debt_count(df, "многодетные", 1) + children_debt_count(df, "многодетные", 0) #общее кол-во многодетных клиентов

#переменные для подсчета доли должников с просрочкой от общего количества клиентов в группе:
got_children_part = '{:.2%}'.format(children_debt_sum / children_debt_total) #доля должников с детьми
childfree_part = '{:.2%}'.format(childfree_debt_sum / childfree_debt_total) #доля должников без детей
large_family_part = '{:.2%}'.format(large_family_debt_sum/ large_family_debt_total) #доля многодетных должников

In [61]:
# посчитаем долю должников, не имеющих детей, к клиентам, не имеющим дететей и не имеющим просрочек платежей. 
a_test = children_debt_count(df, "нет детей", 1) / (children_debt_count(df, "нет детей", 1) + \
                                                    children_debt_count(df, "нет детей", 0)) 

a_test

0.07535266179910682

In [62]:
#для наглядности занесем полученные данные в отдельную таблицу

data=data = [['нет детей', childfree_debt_total, childfree_debt_sum, childfree_part], 
             ['есть дети', children_debt_total, children_debt_sum, got_children_part],
            ['многодетные', large_family_debt_total, large_family_debt_sum, large_family_part]]
columns = ['наличие детей', 'всего', 'дожники', 'доля должников']
table = pd.DataFrame(data = data, columns = columns )

table

,наличие детей,всего,дожники,доля должников
0,нет детей,14107,1063,7.54%
1,есть дети,6861,638,9.30%
2,многодетные,456,39,8.55%


**Вывод**

Не имеющих детей клиентов в два раза больше, чем клиентов с детьми. Этим объясняется большее количество должников среди них. Однако, из полученных данных видно, что доля должников, имеющих детей, из группы клиентов, имеющих детей, больше доли должников, у которых детей нет. В связи с этим, мы можем утверждать о наличии взаимосвязи между наличием детей и возвратом кредита в срок - наличие детей увеличивает вероятность просрочки платежей.  

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [63]:
# создадим функцию, которая будет возвращать количество должников, состоящих в официальном браке
def family_debt_count(df, status, debt): 
    debt_list = df[(df['marriage'] == status)&(df['debt']==debt)]
    debt_list_count = debt_list.loc[:, 'debt'].count()
    return debt_list_count
    

In [64]:
#создадим переменные, которых будут содержаться данные о количестве должников по наличию/отсутствую брака

#переменные для подсчета количества клиентов с просрочкой, состоящих в браке:
married_debt_sum = family_debt_count(df, 'оф. брак', 1) #должники, состоящие в браке
married_debt_total = family_debt_count(df, 'оф. брак', 1) + family_debt_count(df, 'оф. брак', 0) #общее количество клиентов, состоящих в браке

#переменные для подсчета количества должников, не состоящих в официальном браке:
unmarried_debt_sum = family_debt_count(df, 'не сост в оф. браке', 1) #должники, не состоящие в официальном браке
unmarried_debt_total = family_debt_count(df, 'не сост в оф. браке', 1) + family_debt_count(df, 'не сост в оф. браке', 0)#общее количество клиентов, не состоящих в браке

#переменные для подсчета доли должников в группе:
married_debt_part = '{:.2%}'.format(married_debt_sum / married_debt_total)
unmarried_debt_part = '{:.2%}'.format(unmarried_debt_sum / unmarried_debt_total)


In [65]:
# создаем отдельную таблицу
data=data = [['Состоят в официальном браке', married_debt_total, married_debt_sum, married_debt_part], 
             ['Не состоят в официальном браке', unmarried_debt_total, unmarried_debt_sum, unmarried_debt_part]]
columns = ['Брак', 'всего', 'дожники', 'доля должников']
table = pd.DataFrame(data = data, columns = columns )

table

,Брак,всего,дожники,доля должников
0,Состоят в официальном браке,12344,931,7.54%
1,Не состоят в официальном браке,9127,810,8.87%


**Вывод**

Большинство клиентов состоят в официальном браке. Количество должников среди состоящих в официальном браке чуть выше, чем среди не состоящих. В то же время, не так значительно. Из этого можно сделать вывод, что количество должников в каждой группе примерно одинаково. Однако, доля должников, не состоящих в официальном браке, выше доли должников состоящих. Поэтому можно сделать вывод о том, что отсутствие действующего брака увеличивает вероятность просрочки. 

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [66]:
# сгруппируем датасет с должниками и сортируем его по уровню дохода
df_debt = df[df['debt'] == 1] 
df_debt_sorted = df_debt.groupby(['income_group'])['debt'].count().sort_values(ascending=False)

#сгруппируем основной датасет по группам и получим данные об общем количестве клиентов по каждой группе 
df_debt_income_total = df.groupby(['income_group'])['debt'].count().sort_values(ascending=False)

#из полученных данных сделаем два отдельных ДатаФрейма
data_debt_income = pd.DataFrame(data = df_debt_sorted, columns = ['debt'])
data_debt_total = pd.DataFrame(data = df_debt_income_total, columns = ['debt'])

#теперь соединим их методом merge()
data_inc = data_debt_total.merge(data_debt_income, on='income_group', how = 'right')

#поменяем названия столбцов
data_inc.set_axis(['total', 'debt'], axis = 'columns', inplace=True)

#новая таблица готова
data_inc

,total,debt
income_group,,
предсредний класс,14195,1193
выше бедности,4388,350
средний класс,2591,180
верхний средний класс,197,12
бедные,75,4
состоятельные,25,2


In [67]:
# создадим функцию для подсчета процентного соотношения количества должников к общему количеству в группе 
def debt_ratio(row):
    debt = row['debt']
    total = row['total']
    ratio = debt / total
    return '{:.2%}'.format(ratio)

In [68]:
# применим функцию к таблице, создав новый столбец ['ratio']
data_inc['ratio'] = data_inc.apply(debt_ratio, axis=1)

# сортируем таблицу по новому столбцу ['ratio'] и посмотрит на результаты
data_inc.sort_values(by='ratio', ascending=False)

,total,debt,ratio
income_group,,,
предсредний класс,14195,1193,8.40%
состоятельные,25,2,8.00%
выше бедности,4388,350,7.98%
средний класс,2591,180,6.95%
верхний средний класс,197,12,6.09%
бедные,75,4,5.33%


**Вывод**

Самыми внушающими доверие клиентами являются представители среднего, верхнего среднего класса, и, как это неудивительно, бедные. Предсредний класс, наоборот, оказался менее благонадежными из всех. 

В целом, это легко поддается объяснению. Во-первых, представители среднего класса - это либо владельцы устойчивого бизнеса и его руководящий персонал, либо востребованные высококвалифицированные специалисты, имеющие регулярную заработную плату. Иными словами, имеющие весомый стабильный доход. Большая часть же предсреднего класса - это как правило торговцы и мелкие предприниматели, доход которых часто варируется по синусоиде. 

Благонадежность бедных, на наш взгляд, заключается в следующем. Во-первых, банк им одобряет лишь небольшие суммы. Во-вторых, бедные прекрасно понимают, что взять кредит для них - это риск, и в случае просрочки они испортят себе кредитную историю, что в последующем станет препятствием в получении нового. В-третьих, бедные хорошо осведомлены, что за просрочки платежей банк начисляет неустойку, и в конечном итоге денег на возврат кредита будет потрачено еще больше. Таким образом, бедные в своем большинстве, более мотивированы и организованы в выплате кредита, а небольшая сумма ежемесячного платежа позволяет им выплачивать кредит относительно комфортно, чем и объясняется самый маленький процент должников среди них.

Относительно неблагонадежности "состоятельных" хотим отметить, что в нашей выборке их представлено всего 25 человек (0,1% от общего количества всех клиентов). Если их включить в верхний средний класс, результаты исследования практически не изменяться. Средний класс все равно окажется более благонадежным чем предсредний. Однако, полагаем, что кредитная благонадежность состоятельных людей заслуживает отдельного исследования.   



- Как разные цели кредита влияют на его возврат в срок?

In [69]:
#создадим функци, которая вернет нам количество должников по определенной цели кредита
def purpose_debt_count(df, purpose, debt):
    debt_list = df[(df['purpose_group'] == purpose)&(df['debt']==debt)]
    debt_list_count = debt_list.loc[:, 'debt'].count()
    return debt_list_count
    

In [70]:
#сгруппируем данные по целевым группам и создадим отдельный ДатаФрейм
df_purpose_group = df.groupby(['purpose_group'])['debt'].count().sort_values(ascending=False)
data_purpose_total = pd.DataFrame(data = df_purpose_group, columns = ['debt']) 

#используя функцию для подсчета должников, создадим ДатаФрейм из должников с просрочкой
data=data = [ ['жилье', purpose_debt_count(df, 'жилье', 1)], 
             ['автомобиль', purpose_debt_count(df, 'автомобиль', 1)], 
             ['образование', purpose_debt_count(df, 'образование', 1)], 
             ['свадьба', purpose_debt_count(df, 'свадьба', 1)]]
columns = ['purpose_group', 'debt']
data_purpose_debt = pd.DataFrame(data = data, columns=columns)

#объединим полученные таблицы
data_purpose_final = data_purpose_total.merge(data_purpose_debt, on = 'purpose_group', how = 'right')

#поменяем названия столбцов
data_purpose_final.set_axis(['purpose_group', 'total', 'debt'], axis = 'columns', inplace=True)

#создадим новый столбец с подсчетом доли должников в группе
data_purpose_final['ratio'] = (data_purpose_final['debt'] / data_purpose_final['total'])*100

#отсортируем таблицу по убыванию размера доли должников
data_purpose_final.sort_values(by='ratio', ascending=False)

,purpose_group,total,debt,ratio
1,автомобиль,4308,403,9.354689
2,образование,4014,370,9.217738
3,свадьба,2335,186,7.965739
0,жилье,10814,782,7.231367


**Вывод**

Самая популярная цель кредитования - приобретение жилья. Вполне ожидаемо, что больше всего должников будут именно из этой группы. Однако, процентное соотношение по должникам из них самое низкое. Полагаем, что это связано с тем, что решение о приобретениии жилья люди принимают более осознано, когда уверены в стабильности своего дохода. Иными словами, когда чувтсвуют, что они прочно стоят на ногах. 

Покупатели автомобилей менее благонадежны в этом плане, так как автомобиль чаще приобретается в подтверждение своего мнимого социального статуса, нежели просто для перемещения на расстоянии. 

Кредиты на образование берут студенты, у которых нет стабильного дохода, поэтому и должников среди них также весомое количество. 

Примечательно, что соотношение "женатиков" проходит практически по медиане между "домоседами" и "студентами". Можно предположить, что они представляют собой усредненный вариант между людьми, прочно стоящими на ногах, и бедными студентами, которыми нечем оплатить учебу. 

## Шаг 4. Общий вывод

Начнем с того, что большая часть клиентов не имеют проблем с выплатами кредита. Лица, допустившие просрочку, составляют всего 8% от общего числа должников. Из этого следует, что подавляющее большинство клиентов лояльно настроены к банку и избегают появления просрочки. 

Мы выяснили, что доля просрочивших должников, имеющих детей, от общего числа клиентов с детьми, выше, чем аналогичная доля среди клиентов, детей не имеющих. Логично предположить, что дети - это дополнительное финансовое бремя для их родителей, нередко весьма существенное. Поэтому вполне ожидаемо, что имеющие детей клиенты могут допустить просрочку. В то же время, разница в процентном соотношении не так значительна. Исходя из этого, ответ на первый вопрос можно сформулировать так: "Наличие детей создает некритичную вероятность появления просрочки". 

По второму вопросу, изучив данные о семейном положении, мы пришли к выводу, что информация в них представлена не совсем в корректном виде. Как отличить "неженатого клиента" от "клиента в разводе" или "клиента-вдовца", не говоря уже о том, что действующее семейное законодательство не предусматривает такого понятия как "гражданский брак". По этой причине мы приняли решение разделить данные на две категории: "Лица, состоящие в официальном браке" и "Лица, не состоящие в официальном браке". Получив данные, и проанализировав их тем же методом, что и с детьми, мы установили, что доля должников с просрочкой, состоящих в браке, ниже чем должников, не состоящих в браке. Однако, опять же незначительно. Из этого мы пришли к выводу, что наличие официального брака некритично снижает вероятность появления просрочки. 

Для ответа на третий вопрос, мы разделили клиентов по социальным классам. Было выявлено, что менее благонадежными клиентами, являются представители предсреднего класса. Бедные, наоборот, имеют самое маленькое процентное соотношение должников, ввиду своей мотивированности. При этом даже состоятельные допускают просрочки. Из этого мы делаем вывод, что на вероятность просрочки влияет не столько уровень дохода, сколько его стабильность и организованность клиентов. 

По вопросу о целях кредитования, меньше всего просрочек среди клиентов, приобретающих жилье. Больше всего просрочек, среди автовладельцев. Покупка жилья - дорогое приобретение. Редко кому удается накопить на него, минуя ипотеку. Альтернатива только одна - брать жилье в аренду, при этом по своей цене данные варианты равнозначны. Поэтому брать кредит на жилье - больше вынужденная мера для многих людей, к которой они подходят осознано. Покупка автомобиля, да еще и в кредит - это статусное приобретение, которое люди совершают не всегда осознано, чем и объясняют наличие большей доли от должников. . 